# XGBoost

In [3]:
!pip install xgboost

  Using cached xgboost-2.1.0-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.0-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (2.1 MB)


In [8]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/house_price/train/cluster_0_important_features.csv'
data = pd.read_csv(data_path)

# 選択された特徴量
important_features = [
    'OverallQual_TotalArea', 'OverallConditionArea', 'Age', 'BsmtUnfSF', 'YearBuilt', 
    'LotArea', 'log_LotArea', 'QualityScore', 'TotalArea', 'GrLivArea_OverallQual', 
    'RemodelAge', 'OverallQual_GrLivArea', 'GarageYrBlt', 'YearRemodAdd', 
    'TotalRmsAbvGrd_OverallCond', 'TotalBsmtSF_OverallQual'
]

# 特徴量と目的変数を分ける
X = data[important_features]
y = data['SalePrice']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoostモデルの定義
xgboost_model = xgb.XGBRegressor(random_state=42)

# ハイパーパラメータのグリッド
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# グリッドサーチの設定
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=0)

# グリッドサーチの実行
grid_search.fit(X_train, y_train)

# 最適なハイパーパラメータの表示
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation MAE: {-grid_search.best_score_:.4f}")

# 最適なモデルを取得
best_model = grid_search.best_estimator_

# テストデータを使った予測
y_pred = best_model.predict(X_test)

# 評価指標の計算
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# 結果の表示
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2): {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# 最適なモデルの保存
model_path = os.path.join('/Users/hayakawakazue/Downloads/house_price/model', 'xgboost_Cluster_0711.joblib')
joblib.dump(best_model, model_path)
print(f"最適なXGBoostモデルが {model_path} に保存されました。")

# 実際の値の平均を計算
mean_actual_value = np.mean(y_test)
print(f"Mean Actual Value: {mean_actual_value}")

# 誤差のパーセンテージを計算
error_percentage = (mae / mean_actual_value) * 100
print(f"Mean Absolute Percentage Error (MAPE): {error_percentage:.2f}%")


Best parameters found: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.6}
Best cross-validation MAE: 12674.3372
Mean Absolute Error (MAE): 12093.7756
Mean Squared Error (MSE): 328254398.2927
R-squared (R2): 0.6765
Root Mean Squared Error (RMSE): 18117.7923
最適なXGBoostモデルが /Users/hayakawakazue/Downloads/house_price/model/xgboost_Cluster_0711.joblib に保存されました。
Mean Actual Value: 137296.6384180791
Mean Absolute Percentage Error (MAPE): 8.81%


# LightGBMの例

# PCAで次元削減しLightGBMモデルをトレーニングする

In [1]:
# 必要なライブラリのインポート
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# PCA後のデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/house_price/train/reduced_top_30_features_pca.csv'
data = pd.read_csv(data_path)

# 特徴量と目的変数を分ける
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ハイパーパラメータの設定
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'feature_fraction': [0.7, 0.8, 0.9]
}

# LightGBMモデルの初期化
lgb_estimator = lgb.LGBMRegressor(objective='regression', boosting_type='gbdt')

# GridSearchCVの設定
grid = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=10, verbose=0)

# グリッドサーチの実行
grid.fit(X_train, y_train)

# 最適パラメータの表示
print(f"Best parameters found: {grid.best_params_}")

# 最適パラメータでのモデル訓練
best_params = grid.best_params_

# LightGBMのデータセット作成
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'feature_fraction': best_params['feature_fraction']
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

# 予測
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 評価
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = mse ** 0.5
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R2: {r2}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000133 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 10
[LightGBM] [Info] Start training from score 178610.095623
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Info] Auto-choosing co